In [ ]:
voice = pd.read_csv('https://cainvas-static.s3.amazonaws.com/media/user_data/cainvas-admin/voice.csv')
voice

In [ ]:
voice['label'].value_counts()

In [ ]:
#correlation between various attributes
corr = voice.corr(numeric_only=True)
corr

In [ ]:
 # removing columns where corelation is more than .95

final_columns = list(voice.columns)    # maintaining a temporary list to remove columns from

for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i, j] >= 0.95:
            if list(voice.columns)[j] in final_columns:
                final_columns.remove(list(voice.columns)[j])

voice = voice[final_columns]

voice

In [ ]:
voice.describe()

In [ ]:
input_columns = list(voice.columns[:-1])
output_columns = ['male', 'female']    # column names to be used after one-hot encoding
print("Number of input columns: ", len(input_columns))
print("Number of output columns: ", len(output_columns))

In [ ]:
# One hot encoding the labels
y = pd.get_dummies(voice.label)
print(y)

for x in output_columns:
    voice[x] = y[x]

In [ ]:
voice

In [ ]:
# Splitting into train, val and test set -- 80-10-10 split

# 80-20 split
train_df, val_test_df = train_test_split(voice, test_size = 0.2)

# Then split the 20% into half
val_df, test_df = train_test_split(val_test_df, test_size = 0.5)

print("Number of samples in...")
print("Training set: ", len(train_df))
print("Validation set: ", len(val_df))
print("Testing set: ", len(test_df))

In [ ]:
# Splitting into X (input) and y (output)

Xtrain, ytrain = np.array(train_df[input_columns]), np.array(train_df[output_columns])

Xval, yval = np.array(val_df[input_columns]), np.array(val_df[output_columns])

Xtest, ytest = np.array(test_df[input_columns]), np.array(test_df[output_columns])

In [ ]:
# Each feature has a different range.
# Using min_max_scaler to scale them to values in the range [0,1].

min_max_scaler = MinMaxScaler()

# Fit on training set alone
Xtrain = min_max_scaler.fit_transform(Xtrain)

# Use it to transform val and test input
Xval = min_max_scaler.transform(Xval)
Xtest = min_max_scaler.transform(Xtest)

voice.describe()

In [ ]:
model = tf.keras.Sequential([
    layers.Dense(16, activation = 'relu', input_shape = Xtrain[0].shape),
    layers.Dense(8, activation = 'relu'),
    layers.Dense(len(output_columns), activation = 'softmax')
])

model.compile(optimizer = tf.keras.optimizers.Adam(0.01), loss = tf.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

callbacks = [EarlyStopping(monitor='val_loss', patience = 5),    # stop if the val_loss metric doesn;t increase for 5 epochs continuously.
             ModelCheckpoint('gender_recognition_voice.keras', save_best_only=True)]   # save the best model yet

model.summary()

In [ ]:
history = model.fit(Xtrain, ytrain, validation_data = (Xval, yval), epochs=32, callbacks=callbacks)

In [ ]:
model.load_weights('gender_recognition_voice.keras')    # Load the weights of the best model

model.evaluate(Xtest, ytest)

In [ ]:
def plot(history, variable, variable1):
    plt.plot(range(len(history[variable])), history[variable])
    plt.plot(range(len(history[variable1])), history[variable1])
    plt.title(variable)
    plt.legend([variable, variable1])

In [ ]:
plot(history.history, "accuracy", "val_accuracy")

In [ ]:
plot(history.history, "loss", "val_loss")


In [ ]:
# Pick random test sample
i = random.randint(0, len(test_df)-1)

model_output = model.predict(Xtest[i].reshape(1, -1))[0]
pred = np.argmax(model_output)

# show predicted output
print ("\nModel predicted the gender: ", output_columns[pred])

# actual output
print("Actual gender: ", output_columns[np.argmax(ytest[i])], "with probability", model_output[pred])

In [ ]:
import librosa
import pydub
import soundfile as sf
from scipy.stats import skew, kurtosis
print("Imported librosa, pydub, soundfile, and scipy.stats.")

In [ ]:
import numpy as np
import librosa
import librosa.display
import scipy.stats
import soundfile as sf # For loading diverse audio formats
import os

def extract_features_and_scale(file_path, scaler, input_columns, sr=22050):
    """
    Extracts 17 audio features from an audio file and scales them using a pre-fitted MinMaxScaler.

    Args:
        file_path (str): Path to the audio file.
        scaler (MinMaxScaler): A pre-fitted MinMaxScaler object.
        input_columns (list): List of 17 feature names in the expected order.
        sr (int): Target sample rate for audio processing.

    Returns:
        np.ndarray: A (1, 17) array of scaled features.
    """
    try:
        # 1. Load Audio, limit duration to 5s for faster processing
        y, sr = librosa.load(file_path, sr=sr, mono=True, duration=5.0)
    except Exception as e:
        print(f"Error loading audio file {file_path}: {e}")
        # Return a zero array of features if audio cannot be loaded or is corrupted
        return np.zeros((1, len(input_columns)))

    # Initialize feature dictionary to store calculated values
    features = {}

    # --- Spectral features (meanfreq, sd, median, Q25, Q75, IQR, sp.ent, sfm, mode, skew, kurt) ---
    # Compute short-time Fourier transform (STFT)
    n_fft = 2048 # Standard FFT window size
    hop_length = 512 # Standard hop length
    D = librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length)
    S_magnitude = np.abs(D)

    # Calculate power spectrum (magnitude squared) for spectral moments
    S_power = S_magnitude**2

    # Average power spectrum over time frames to get a single spectral profile
    S_avg_power = np.mean(S_power, axis=1)

    # Frequencies associated with the STFT bins (in Hz)
    freqs_hz = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

    # The dataset's 'kHz' values for meanfreq, sd, median, Q25, Q75, IQR, mode are much smaller
    # than raw kHz from librosa (max ~11 kHz). This implies a scaling factor was applied.
    # Max 'meanfreq' in dataset is ~0.25 kHz. Max possible librosa freq in kHz is sr/2/1000 = 11.025.
    # So, apply a global scaling factor to bring librosa's frequencies into the dataset's 'kHz' scale.
    # Derived factor: dataset_max_meanfreq / librosa_max_freq_khz (approx 0.25 / 11.025 = 0.0226)
    global_freq_scaling_factor = 0.02277 # Based on 0.251124 (dataset max) / (22050/2/1000)

    # Apply this scaling to freqs_hz before converting to the dataset's 'kHz'
    freqs_khz_scaled = (freqs_hz / 1000.0) * global_freq_scaling_factor

    # Ensure S_avg_power is not all zeros before normalization to avoid division by zero
    sum_S_avg_power = np.sum(S_avg_power)
    if sum_S_avg_power == 0:
        # Handle cases with no audible signal (assign defaults)
        S_prob = np.zeros_like(S_avg_power)
    else:
        S_prob = S_avg_power / sum_S_avg_power # Normalize to represent a probability distribution

    # Only consider positive normalized frequencies (and their corresponding probability)
    positive_freq_indices = freqs_khz_scaled > 0
    freqs_pos_khz_scaled = freqs_khz_scaled[positive_freq_indices]
    S_prob_pos = S_prob[positive_freq_indices]

    if np.sum(S_prob_pos) == 0 or len(freqs_pos_khz_scaled) == 0:
        # If no energy in positive frequencies or no positive frequencies, assign defaults
        features['meanfreq'] = 0.0
        features['sd'] = 0.0
        features['median'] = 0.0
        features['Q25'] = 0.0
        features['Q75'] = 0.0
        features['IQR'] = 0.0
        features['skew'] = 0.0
        features['kurt'] = 0.0
        features['mode'] = 0.0
    else:
        # Mean frequency (in dataset's 'kHz')
        features['meanfreq'] = np.sum(freqs_pos_khz_scaled * S_prob_pos)

        # Standard deviation of frequency (in dataset's 'kHz')
        features['sd'] = np.sqrt(np.sum((freqs_pos_khz_scaled - features['meanfreq'])**2 * S_prob_pos))

        # Quantiles (Q25, median, Q75) and IQR from the cumulative distribution
        cumulative_S_prob_pos = np.cumsum(S_prob_pos)

        # Ensure we don't access out of bounds for quantiles
        features['Q25'] = freqs_pos_khz_scaled[np.where(cumulative_S_prob_pos >= 0.25)[0][0]] if len(np.where(cumulative_S_prob_pos >= 0.25)[0]) > 0 else 0.0
        features['median'] = freqs_pos_khz_scaled[np.where(cumulative_S_prob_pos >= 0.5)[0][0]] if len(np.where(cumulative_S_prob_pos >= 0.5)[0]) > 0 else 0.0
        features['Q75'] = freqs_pos_khz_scaled[np.where(cumulative_S_prob_pos >= 0.75)[0][0]] if len(np.where(cumulative_S_prob_pos >= 0.75)[0]) > 0 else 0.0
        features['IQR'] = features['Q75'] - features['Q25']

        # Skewness and Kurtosis of the power spectrum distribution (weighted by probability)
        mean_freq_val = features['meanfreq']
        variance = np.sum((freqs_pos_khz_scaled - mean_freq_val)**2 * S_prob_pos)
        std_dev = np.sqrt(variance)

        if std_dev > 1e-9:
            m3 = np.sum((freqs_pos_khz_scaled - mean_freq_val)**3 * S_prob_pos)
            features['skew'] = m3 / (std_dev**3)

            m4 = np.sum((freqs_pos_khz_scaled - mean_freq_val)**4 * S_prob_pos)
            features['kurt'] = (m4 / (std_dev**4)) - 3 # Excess kurtosis
        else:
            features['skew'] = 0.0
            features['kurt'] = 0.0

        # Mode: frequency (in dataset's 'kHz') with the highest magnitude
        features['mode'] = freqs_pos_khz_scaled[np.argmax(S_prob_pos)]

    # Spectral Entropy (sp.ent) - unitless ratio
    eps = 1e-10 # Small epsilon to avoid log(0)
    spectral_entropy_val = -np.sum(S_prob * np.log2(S_prob + eps)) # Use S_prob for full spectrum
    max_entropy = np.log2(len(S_prob)) if len(S_prob) > 1 else 0
    features['sp.ent'] = spectral_entropy_val / max_entropy if max_entropy > 0 else 0.0

    # Spectral Flatness Measure (sfm) - unitless ratio
    sfm_frames = librosa.feature.spectral_flatness(S=S_magnitude, hop_length=hop_length, power=1) # power=1 for magnitude
    features['sfm'] = np.mean(sfm_frames) if sfm_frames.size > 0 else 0.0


    # --- Fundamental frequency (F0) related features (meanfun, minfun, maxfun, modindx) --- #
    # Using pYIN for pitch detection (robust F0 estimation)
    f0, voiced_flag, voiced_probs = librosa.pyin(y=y, sr=sr, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'),
                                                 frame_length=n_fft, hop_length=hop_length)
    voiced_f0 = f0[voiced_flag]

    if len(voiced_f0) == 0:
        features['meanfun'] = 0.0
        features['minfun'] = 0.0
        features['maxfun'] = 0.0
        features['modindx'] = 0.0
    else:
        # Normalizing F0 to match the typical range of 'meanfun' [0.05, 0.25] from dataset
        # Max 'meanfun' in dataset is ~0.237. Max human F0 can be ~500Hz. 500 / 2100 = 0.238
        f0_normalization_factor_fun = 2100.0
        voiced_f0_norm_fun = voiced_f0 / f0_normalization_factor_fun

        features['meanfun'] = np.mean(voiced_f0_norm_fun)
        features['minfun'] = np.min(voiced_f0_norm_fun)
        features['maxfun'] = np.max(voiced_f0_norm_fun)

        if len(voiced_f0_norm_fun) > 1:
            features['modindx'] = np.std(voiced_f0_norm_fun)
        else:
            features['modindx'] = 0.0


    # --- Dominant frequency features (meandom, mindom, maxdom) --- #
    # These are statistics of the spectral peak frequency across frames. (not F0 related here)
    dominant_frequencies_per_frame_khz_raw = [] # Store raw kHz from STFT
    # Iterate through each frame of the magnitude spectrogram to find the dominant frequency
    for t in range(S_magnitude.shape[1]):
        frame_power = S_magnitude[:, t]**2
        if np.sum(frame_power) > 0:
            dominant_idx = np.argmax(frame_power)
            # Use the *original* freqs_khz here, then apply the global scaling
            dominant_frequencies_per_frame_khz_raw.append(freqs_hz[dominant_idx] / 1000.0)

    dominant_frequencies_per_frame_khz_raw = np.array(dominant_frequencies_per_frame_khz_raw)

    if len(dominant_frequencies_per_frame_khz_raw) == 0:
        features['meandom'] = 0.0
        features['mindom'] = 0.0
        features['maxdom'] = 0.0
    else:
        # Now apply the global scaling factor to these dominant frequencies as well
        normalized_dominant_frequencies = dominant_frequencies_per_frame_khz_raw * global_freq_scaling_factor

        features['meandom'] = np.mean(normalized_dominant_frequencies)
        features['mindom'] = np.min(normalized_dominant_frequencies)
        features['maxdom'] = np.max(normalized_dominant_frequencies)

    # Handle potential NaN or Inf values by replacing them with 0 after all calculations
    for key in features:
        if np.isnan(features[key]) or np.isinf(features[key]):
            features[key] = 0.0

    # Ensure the order of extracted features matches the input_columns list
    extracted_features_ordered = [features[col] for col in input_columns]

    # Debugging print: Show raw extracted features before scaling/clipping
    # print(f"--- Extracted features for {os.path.basename(file_path)} (before scaling/clipping) ---")
    for i, col_name in enumerate(input_columns):
        min_val_train = scaler.data_min_[i]
        max_val_train = scaler.data_max_[i]
        val = extracted_features_ordered[i]
        # print(f"  {col_name}: {val:.6f} (Train range: [{min_val_train:.6f}, {max_val_train:.6f}])")
    # print("------------------------------------------------------")

    # Convert to NumPy array and reshape to (1, -1) for MinMaxScaler
    features_array = np.array(extracted_features_ordered).reshape(1, -1)

    # --- Clip features to the range of training data before scaling to prevent extrapolation ---
    if features_array.shape[1] == scaler.data_min_.shape[0]:
        for i, col_name in enumerate(input_columns):
            original_val = features_array[0, i]
            min_val = scaler.data_min_[i]
            max_val = scaler.data_max_[i]

            # Apply clipping
            features_array[0, i] = np.clip(original_val, min_val, max_val)
    else:
        print("Warning: Feature array shape does not match scaler's input dimension. Clipping skipped.")

    # Apply the pre-fitted MinMaxScaler to transform the features
    scaled_features = scaler.transform(features_array)

    return scaled_features

# print("")

In [ ]:
model.load_weights('gender_recognition_voice.keras')
print("Loaded model weights from 'gender_recognition_voice.keras'")

In [ ]:
import warnings
warnings.filterwarnings('ignore') # Suppress warnings from librosa, etc.

# List of user's 'male' recordings to process
recorded_audio_files = ['/justin.wav', '/sahiba.wav', '/pretty_little.wav','/vikram.wav']

for audio_file_path in recorded_audio_files:
    # 1. Extract and scale features from the audio file
    scaled_audio_features = extract_features_and_scale(audio_file_path, min_max_scaler, input_columns)

    # 2. Use the model to predict the gender from the scaled features
    model_prediction = model.predict(scaled_audio_features)[0]

    # 3. Determine the predicted gender
    pred_index = np.argmax(model_prediction)
    predicted_gender = output_columns[pred_index]

    # 4. Print the path of the processed audio file, the predicted gender, and the associated probability
    print(f"\n------------------------------------------------------")
    print(f"Processing audio file: '{audio_file_path}'")
    print(f"Predicted gender: {predicted_gender}")
    print(f"Prediction probabilities (male, female): {model_prediction}")
    print(f"Prediction probability for {predicted_gender}: {model_prediction[pred_index]:.4f}")